In [1]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_pl = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/t20_pl_g.csv')

In [2]:
df_pl = df_pl.sort_values(by=['start_date','match_id','innings','ball']).reset_index(drop=True)

In [3]:
df_pl['phase'] = np.where(df_pl['ball']<7, 'pp', 
                        np.where(df_pl['ball']>=15, 'death',
                            'middle'))

In [4]:
player_list = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/player_list.xlsx',
                           sheet_name='season_03')

player_list.iloc[130:135]

players_list = player_list['Func_Name'].dropna().tolist()

#players_list = players_list.remove('nan')

print(type(players_list))

len(players_list)

<class 'list'>


247

In [5]:
df_h = df_pl[(df_pl.striker.isin(players_list))&(df_pl.bowler.isin(players_list))]

df_h.shape

(100234, 61)

In [6]:
df_h['runs_conceeded'] = df_h['runs_off_bat']+df_h['wides']+df_h['noballs']

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_9975/3799187946.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h['runs_conceeded'] = df_h['runs_off_bat']+df_h['wides']+df_h['noballs']


In [7]:
# head-to-head statistics

In [64]:
h2h_stats = df_h.groupby(['striker','bowler']).agg(
    
    runs_scored = ('runs_off_bat','sum'),
    runs_conceeded = ('runs_conceeded','sum'),
     
    balls_faced = ('is_faced_by_batter' ,'sum'),
    balls_bowled = ('islegal','sum'),
    
    outs = ('isBowlerWicket','sum')
    
).reset_index()


h2h_stats['bat_avg'] = h2h_stats['runs_scored']/h2h_stats['outs']
h2h_stats['bowl_sr'] = h2h_stats['balls_bowled']/h2h_stats['outs']
h2h_stats['bat_sr'] = 100*h2h_stats['runs_scored']/h2h_stats['balls_faced']
h2h_stats['bowl_eco'] = 6*h2h_stats['runs_conceeded']/h2h_stats['balls_bowled']




#h2h_stats.sample(10)


In [65]:
h2h_stats['bat_avg'] = h2h_stats.apply(
    lambda row: row['balls_faced']*1.5 if np.isinf(row['bat_avg']) else row['bat_avg'], axis=1
)

h2h_stats['bowl_sr'] = h2h_stats.apply(
    lambda row: row['balls_bowled']*1.5 if np.isinf(row['bowl_sr']) else row['bowl_sr'], axis=1
)

h2h_stats['bowl_eco'] = h2h_stats.apply(
    lambda row: 36 if np.isinf(row['bowl_eco']) else row['bowl_eco'], axis=1
)

bat_fig = 3*140+0.5*35
bowl_fig = 27

h2h_stats['bat'] = (3*h2h_stats['bat_sr']+0.5*h2h_stats['bat_avg'])/bat_fig
h2h_stats['bat'] = np.sqrt(h2h_stats['bat'])
h2h_stats['bowl'] = 2-h2h_stats['bat']
#h2h_stats['bowl'] = bowl_fig/(h2h_stats['bowl_sr']+h2h_stats['bowl_eco'])

h2h_stats['bat'] = np.where(h2h_stats['balls_faced']<20, 0, h2h_stats['bat'])
h2h_stats['bowl'] = np.where(h2h_stats['balls_bowled']<20, 0, h2h_stats['bowl'])

#h2h_stats['bat'] = np.sqrt(h2h_stats['bat'])
#h2h_stats['bowl'] = np.sqrt(h2h_stats['bowl'])

#h2h_stats['bat'] = h2h_stats['bat']/h2h_stats['bat'].mean()
#h2h_stats['bowl'] = h2h_stats['bowl']/h2h_stats['bowl'].mean()

In [66]:
h2h_stats[h2h_stats.balls_faced>=20].bowl.describe()

count    1342.000000
mean        1.000447
std         0.172815
min         0.000000
25%         0.907788
50%         1.008799
75%         1.102592
max         1.538811
Name: bowl, dtype: float64

In [67]:
h2h_stats['bat'] = np.where(h2h_stats['bowl']==0, 2, h2h_stats['bat'])
h2h_stats['bowl'] = np.where(h2h_stats['bat']==0, 2, h2h_stats['bowl'])

h2h_stats['a'] = 2/(h2h_stats['bat']+h2h_stats['bowl'])

h2h_stats['bat_a'] = h2h_stats['bat']*h2h_stats['a']
h2h_stats['bowl_a'] = h2h_stats['bowl']*h2h_stats['a']

h2h_stats['h2h_factor_bat'] = np.where(h2h_stats['balls_faced']<20, 0, h2h_stats['bat_a'])
h2h_stats['h2h_factor_bowl'] = np.where(h2h_stats['balls_bowled']<20, 0, h2h_stats['bowl_a'])


#h2h_stats['h2h_factor_bat'] = 

h2h_stats[h2h_stats.striker=='TM Head'].sort_values(by='runs_scored', ascending=False).round(2)

In [68]:
h2h_stats_trimmed = h2h_stats[(h2h_stats.balls_bowled>=20)&(h2h_stats.balls_faced>=20)]\
                            [['striker','bowler','runs_scored','balls_bowled',
                          'outs','h2h_factor_bat','h2h_factor_bowl']]

h2h_stats_trimmed.shape

(1329, 7)

h2h_stats_trimmed = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/h2h_stats.csv'
                           )

In [69]:
h2h_stats_trimmed[h2h_stats_trimmed.outs==0].sort_values(by='h2h_factor_bat').tail(10).round(3)

,striker,bowler,runs_scored,balls_bowled,outs,h2h_factor_bat,h2h_factor_bowl
7430,SA Yadav,LH Ferguson,80,36,0,1.259,0.741
3452,KA Pollard,SN Thakur,52,23,0,1.261,0.739
2966,JC Buttler,Yash Dayal,92,40,0,1.268,0.732
7750,SP Narine,MP Stoinis,53,23,0,1.273,0.727
251,AD Russell,MA Starc,55,23,0,1.296,0.704
5911,Q de Kock,PJ Cummins,57,23,0,1.319,0.681
5615,PJ Cummins,SM Curran,53,21,0,1.329,0.671
8712,TM Head,HH Pandya,63,25,0,1.331,0.669
4793,MS Dhoni,JD Unadkat,61,24,0,1.336,0.664
3496,KD Karthik,JD Unadkat,55,20,0,1.386,0.614


In [71]:
h2h_stats_trimmed[h2h_stats_trimmed.striker=='V Kohli'].sort_values(by='h2h_factor_bat').round(3)

,striker,bowler,runs_scored,balls_bowled,outs,h2h_factor_bat,h2h_factor_bowl
8991,V Kohli,Ravi Bishnoi,36,41,1,0.802,1.198
8940,V Kohli,JD Unadkat,34,36,1,0.829,1.171
8909,V Kohli,A Nortje,41,42,2,0.832,1.168
8911,V Kohli,AD Russell,29,30,1,0.834,1.166
8913,V Kohli,AR Patel,51,51,1,0.863,1.137
8943,V Kohli,JR Hazlewood,33,31,1,0.876,1.124
8984,V Kohli,R Ashwin,44,42,1,0.877,1.123
8947,V Kohli,KH Pandya,71,76,0,0.878,1.122
8996,V Kohli,SP Narine,82,77,2,0.882,1.118
8997,V Kohli,Sandeep Sharma,43,38,2,0.895,1.105


In [34]:
h2h_stats_trimmed.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/h2h_stats.csv',
                           index=None)